### Motion Detection

The following code snippet processes a video to detect motion within a specified region of interest (ROI). It uses OpenCV to capture video frames, apply image processing techniques, and save the processed video. This script is useful for applications such as surveillance, where detecting motion within a specific area of a video is required.



#### Installing and Importing Necessary Libraries

In [1]:
# Install necessary packages
!pip install opencv-python
!pip install numpy

In [2]:
# Import necessary libraries
import cv2
import numpy as np

#### Explanation
1. **Install and Import Libraries**:
   - `opencv-python` and `numpy` are installed and imported for video processing and numerical operations.

2. **Function Definition**:
   - `process_video` function processes a video to detect motion in a specified ROI. It captures frames, processes them, and saves the output.

3. **Video Capture and Initialization**:
   - The video is captured using `cv2.VideoCapture`. The first frame is initialized for baseline comparison.

4. **Frame Processing**:
   - Each frame is read and processed. The region of interest (ROI) is defined, converted to grayscale, and blurred to reduce noise.
   - The absolute difference between the baseline and current frame is computed, and a binary threshold is applied.

5. **Morphological Operations**:
   - Dilation and erosion operations are applied to fill gaps and remove small noise in the threshold frame.

6. **Contour Detection**:
   - Contours are detected in the threshold frame. Bounding boxes are drawn around detected objects in the original frame.

7. **Output Video**:
   - The processed frames are written to the output video file using `cv2.VideoWriter`.

8. **Release Resources**:
   - The video capture and writer objects are released, and all OpenCV windows are closed.

This script effectively detects and highlights motion in a specified region of a video, making it suitable for surveillance and monitoring applications.

In [3]:
def process_video(input_path, output_path, roi_coords):
  """
    Process a video to detect motion in a specified region of interest (ROI).

    Parameters:
    input_path (str): Path to the input video file.
    output_path (str): Path to save the processed output video file.
    roi_coords (tuple): Coordinates (x1, y1, x2, y2) defining the ROI.
    """
  # Capture video from file
  video = cv2.VideoCapture(input_path)
  # Initialize the first frame for baseline comparison
  initial_frame = None
  # Define the codec and create VideoWriter object to save output video
  fourcc = cv2.VideoWriter_fourcc(*'mp4v')
  out = cv2.VideoWriter(output_path, fourcc, 20.0, (int(video.get(3)), int(video.get(4))))

  while video.isOpened():
    # Read a frame from the video
    ret, frame = video.read()
    # If no frame is read, end of video is reached
    if not ret:
      break

    # Define the region of interest (ROI)
    roi = frame[roi_coords[1]:roi_coords[3], roi_coords[0]:roi_coords[2]]

    # Convert the ROI to grayscale
    gray_frame = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
    # Apply Gaussian blur to reduce noise and smoothen the image
    blur_frame = cv2.GaussianBlur(gray_frame, (21, 21), 0)

    # The first captured frame is the baseline image
    if initial_frame is None:
      initial_frame = blur_frame
      continue

    # Compute the absolute difference between the baseline and the current frame
    delta_frame = cv2.absdiff(initial_frame, blur_frame)
    # Apply a binary threshold to the delta frame to create a binary image
    threshold_frame = cv2.threshold(delta_frame, 25, 255, cv2.THRESH_BINARY)[1]

    # Create a kernel for morphological operations (dilation and erosion)
    kernel = np.ones((5, 5), np.uint8)
    # Dilate the threshold image to fill gaps in the detected objects
    threshold_frame = cv2.dilate(threshold_frame, kernel, iterations=2)
    # Erode the threshold image to remove small noise
    threshold_frame = cv2.erode(threshold_frame, kernel, iterations=2)

    # Find contours in the threshold frame
    contours, _ = cv2.findContours(threshold_frame, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Filter and merge contours
    for c in contours:
      if cv2.contourArea(c) < 1000:
          continue
      # Get the bounding box coordinates of the contour
      (x, y, w, h) = cv2.boundingRect(c)
      # Draw a rectangle around the detected object in the original frame
      cv2.rectangle(frame, (roi_coords[0] + x, roi_coords[1] + y), (roi_coords[0] + x + w, roi_coords[1] + y + h), (0, 255, 0), 2)
    # Write the processed frame to the output video file
    out.write(frame)
  # Release the video capture and writer objects
  video.release()
  out.release()
  # Close all OpenCV windows
  cv2.destroyAllWindows()

In [4]:
# Define the ROI (Region of Interest (Main Street)) coordinates (x1, y1, x2, y2)
roi_coords = (0, 300, 1040, 600)
# Process the video
input_video_path = '/content/drive/MyDrive/ISP_Final/Ex_1/Traffic_Laramie_1.mp4'
output_video_path = '/content/Processed_Traffic_Laramie_1.mp4'
process_video(input_video_path, output_video_path, roi_coords)